<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [12]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Metrics,Accuracy,BalancedAccuracy,CohenKappa,GeometricMean
from river.evaluate import progressive_val_score

In [13]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/latest/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [14]:
from river.ensemble import BaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 70.77%, BalancedAccuracy: 67.47%, GeometricMean: 66.80%, CohenKappa: 34.45%
[2,000] Accuracy: 70.29%, BalancedAccuracy: 66.94%, GeometricMean: 66.42%, CohenKappa: 32.37%
[3,000] Accuracy: 71.29%, BalancedAccuracy: 62.56%, GeometricMean: 59.04%, CohenKappa: 26.45%
[4,000] Accuracy: 71.82%, BalancedAccuracy: 61.73%, GeometricMean: 56.61%, CohenKappa: 25.72%
[5,000] Accuracy: 72.67%, BalancedAccuracy: 62.70%, GeometricMean: 57.67%, CohenKappa: 27.97%
[6,000] Accuracy: 73.18%, BalancedAccuracy: 63.95%, GeometricMean: 59.71%, CohenKappa: 30.34%
[7,000] Accuracy: 74.10%, BalancedAccuracy: 65.01%, GeometricMean: 61.06%, CohenKappa: 32.60%
[8,000] Accuracy: 73.83%, BalancedAccuracy: 64.93%, GeometricMean: 61.02%, CohenKappa: 32.39%
[9,000] Accuracy: 73.95%, BalancedAccuracy: 64.68%, GeometricMean: 60.30%, CohenKappa: 32.20%
[10,000] Accuracy: 74.11%, BalancedAccuracy: 64.59%, GeometricMean: 60.03%, CohenKappa: 32.14%
[11,000] Accuracy: 74.56%, BalancedAccuracy: 64.71%, Geomet

Accuracy: 74.68%, BalancedAccuracy: 66.38%, GeometricMean: 62.53%, CohenKappa: 35.78%

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/latest/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [15]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 76.48%, BalancedAccuracy: 73.05%, GeometricMean: 72.38%, CohenKappa: 46.29%
[2,000] Accuracy: 78.04%, BalancedAccuracy: 71.86%, GeometricMean: 70.18%, CohenKappa: 45.52%
[3,000] Accuracy: 78.53%, BalancedAccuracy: 70.76%, GeometricMean: 68.33%, CohenKappa: 44.29%
[4,000] Accuracy: 78.02%, BalancedAccuracy: 69.92%, GeometricMean: 67.08%, CohenKappa: 42.99%
[5,000] Accuracy: 77.88%, BalancedAccuracy: 69.91%, GeometricMean: 67.10%, CohenKappa: 42.92%
[6,000] Accuracy: 77.81%, BalancedAccuracy: 70.48%, GeometricMean: 68.09%, CohenKappa: 43.60%
[7,000] Accuracy: 78.25%, BalancedAccuracy: 71.20%, GeometricMean: 69.05%, CohenKappa: 44.86%
[8,000] Accuracy: 77.85%, BalancedAccuracy: 70.79%, GeometricMean: 68.56%, CohenKappa: 44.05%
[9,000] Accuracy: 77.75%, BalancedAccuracy: 70.38%, GeometricMean: 67.88%, CohenKappa: 43.57%
[10,000] Accuracy: 77.98%, BalancedAccuracy: 70.44%, GeometricMean: 67.87%, CohenKappa: 43.83%
[11,000] Accuracy: 78.14%, BalancedAccuracy: 70.44%, Geomet

Accuracy: 78.24%, BalancedAccuracy: 72.22%, GeometricMean: 70.38%, CohenKappa: 46.79%

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/latest/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [16]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 73.97%, BalancedAccuracy: 65.66%, GeometricMean: 61.14%, CohenKappa: 34.60%
[2,000] Accuracy: 76.44%, BalancedAccuracy: 66.85%, GeometricMean: 62.41%, CohenKappa: 37.51%
[3,000] Accuracy: 77.69%, BalancedAccuracy: 67.20%, GeometricMean: 62.43%, CohenKappa: 38.80%
[4,000] Accuracy: 77.87%, BalancedAccuracy: 67.58%, GeometricMean: 62.73%, CohenKappa: 39.79%
[5,000] Accuracy: 78.00%, BalancedAccuracy: 68.04%, GeometricMean: 63.45%, CohenKappa: 40.65%
[6,000] Accuracy: 77.98%, BalancedAccuracy: 68.54%, GeometricMean: 64.42%, CohenKappa: 41.35%
[7,000] Accuracy: 78.45%, BalancedAccuracy: 69.42%, GeometricMean: 65.77%, CohenKappa: 42.93%
[8,000] Accuracy: 78.23%, BalancedAccuracy: 69.47%, GeometricMean: 65.95%, CohenKappa: 42.89%
[9,000] Accuracy: 78.24%, BalancedAccuracy: 69.42%, GeometricMean: 65.76%, CohenKappa: 42.93%
[10,000] Accuracy: 78.25%, BalancedAccuracy: 69.37%, GeometricMean: 65.72%, CohenKappa: 42.81%
[11,000] Accuracy: 78.49%, BalancedAccuracy: 69.45%, Geomet

Accuracy: 78.54%, BalancedAccuracy: 71.47%, GeometricMean: 68.89%, CohenKappa: 46.29%

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/latest/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [17]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 74.77%, BalancedAccuracy: 67.28%, GeometricMean: 63.73%, CohenKappa: 37.59%
[2,000] Accuracy: 76.49%, BalancedAccuracy: 66.64%, GeometricMean: 61.95%, CohenKappa: 37.27%
[3,000] Accuracy: 77.89%, BalancedAccuracy: 67.07%, GeometricMean: 61.97%, CohenKappa: 38.83%
[4,000] Accuracy: 77.99%, BalancedAccuracy: 67.64%, GeometricMean: 62.74%, CohenKappa: 40.01%
[5,000] Accuracy: 78.06%, BalancedAccuracy: 67.94%, GeometricMean: 63.20%, CohenKappa: 40.59%
[6,000] Accuracy: 77.93%, BalancedAccuracy: 68.11%, GeometricMean: 63.60%, CohenKappa: 40.72%
[7,000] Accuracy: 78.37%, BalancedAccuracy: 68.79%, GeometricMean: 64.63%, CohenKappa: 42.02%
[8,000] Accuracy: 77.95%, BalancedAccuracy: 68.55%, GeometricMean: 64.42%, CohenKappa: 41.38%
[9,000] Accuracy: 77.90%, BalancedAccuracy: 68.42%, GeometricMean: 64.12%, CohenKappa: 41.26%
[10,000] Accuracy: 78.09%, BalancedAccuracy: 68.56%, GeometricMean: 64.27%, CohenKappa: 41.58%
[11,000] Accuracy: 78.34%, BalancedAccuracy: 68.67%, Geomet

Accuracy: 78.37%, BalancedAccuracy: 70.53%, GeometricMean: 67.31%, CohenKappa: 44.97%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [18]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/latest/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [19]:
from river.ensemble import ADWINBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 81.58%, BalancedAccuracy: 74.02%, GeometricMean: 70.47%, CohenKappa: 53.69%
[2,000] Accuracy: 86.14%, BalancedAccuracy: 82.00%, GeometricMean: 80.97%, CohenKappa: 67.48%
[3,000] Accuracy: 87.23%, BalancedAccuracy: 83.85%, GeometricMean: 83.22%, CohenKappa: 70.30%
[4,000] Accuracy: 88.45%, BalancedAccuracy: 85.52%, GeometricMean: 85.09%, CohenKappa: 73.12%
[5,000] Accuracy: 88.52%, BalancedAccuracy: 85.62%, GeometricMean: 85.19%, CohenKappa: 73.36%
[6,000] Accuracy: 83.41%, BalancedAccuracy: 80.88%, GeometricMean: 80.23%, CohenKappa: 63.61%
[7,000] Accuracy: 80.48%, BalancedAccuracy: 78.94%, GeometricMean: 78.45%, CohenKappa: 58.95%
[8,000] Accuracy: 79.11%, BalancedAccuracy: 78.21%, GeometricMean: 77.89%, CohenKappa: 57.05%
[9,000] Accuracy: 77.32%, BalancedAccuracy: 76.88%, GeometricMean: 76.70%, CohenKappa: 54.05%
[10,000] Accuracy: 75.84%, BalancedAccuracy: 75.62%, GeometricMean: 75.51%, CohenKappa: 51.38%
[11,000] Accuracy: 74.11%, BalancedAccuracy: 73.87%, Geomet

Accuracy: 73.14%, BalancedAccuracy: 72.81%, GeometricMean: 72.55%, CohenKappa: 45.86%

## Leveraging Bagging

In [20]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 73.17%, BalancedAccuracy: 65.54%, GeometricMean: 61.41%, CohenKappa: 33.99%
[2,000] Accuracy: 77.89%, BalancedAccuracy: 71.38%, GeometricMean: 68.40%, CohenKappa: 46.49%
[3,000] Accuracy: 79.96%, BalancedAccuracy: 73.86%, GeometricMean: 71.46%, CohenKappa: 51.53%
[4,000] Accuracy: 81.95%, BalancedAccuracy: 76.14%, GeometricMean: 74.20%, CohenKappa: 56.17%
[5,000] Accuracy: 83.22%, BalancedAccuracy: 78.06%, GeometricMean: 76.53%, CohenKappa: 59.73%
[6,000] Accuracy: 78.45%, BalancedAccuracy: 74.95%, GeometricMean: 73.60%, CohenKappa: 52.08%
[7,000] Accuracy: 75.97%, BalancedAccuracy: 74.10%, GeometricMean: 73.35%, CohenKappa: 49.27%
[8,000] Accuracy: 74.92%, BalancedAccuracy: 73.91%, GeometricMean: 73.49%, CohenKappa: 48.40%
[9,000] Accuracy: 74.24%, BalancedAccuracy: 73.75%, GeometricMean: 73.52%, CohenKappa: 47.79%
[10,000] Accuracy: 73.60%, BalancedAccuracy: 73.38%, GeometricMean: 73.27%, CohenKappa: 46.88%
[11,000] Accuracy: 72.16%, BalancedAccuracy: 71.94%, Geomet

Accuracy: 73.68%, BalancedAccuracy: 73.62%, GeometricMean: 73.61%, CohenKappa: 47.23%

## AdaptiveRandomForest

In [21]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 79.68%, BalancedAccuracy: 73.95%, GeometricMean: 71.92%, CohenKappa: 51.16%
[2,000] Accuracy: 80.14%, BalancedAccuracy: 74.01%, GeometricMean: 71.47%, CohenKappa: 52.07%
[3,000] Accuracy: 80.46%, BalancedAccuracy: 73.82%, GeometricMean: 70.97%, CohenKappa: 52.12%
[4,000] Accuracy: 81.75%, BalancedAccuracy: 75.04%, GeometricMean: 72.38%, CohenKappa: 54.82%
[5,000] Accuracy: 82.30%, BalancedAccuracy: 75.89%, GeometricMean: 73.44%, CohenKappa: 56.48%
[6,000] Accuracy: 77.68%, BalancedAccuracy: 72.96%, GeometricMean: 70.41%, CohenKappa: 49.13%
[7,000] Accuracy: 75.70%, BalancedAccuracy: 73.29%, GeometricMean: 72.03%, CohenKappa: 48.14%
[8,000] Accuracy: 74.27%, BalancedAccuracy: 73.15%, GeometricMean: 72.61%, CohenKappa: 46.94%
[9,000] Accuracy: 73.50%, BalancedAccuracy: 73.05%, GeometricMean: 72.86%, CohenKappa: 46.33%
[10,000] Accuracy: 72.59%, BalancedAccuracy: 72.44%, GeometricMean: 72.40%, CohenKappa: 44.94%
[11,000] Accuracy: 70.27%, BalancedAccuracy: 70.22%, Geomet

Accuracy: 71.09%, BalancedAccuracy: 71.09%, GeometricMean: 71.09%, CohenKappa: 42.11%

## StreamingRandomPatches
---
We set the drift and warning detection options

In [22]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier
from river.drift import ADWIN

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)

metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 89.29%, BalancedAccuracy: 85.81%, GeometricMean: 85.17%, CohenKappa: 74.82%
[2,000] Accuracy: 91.75%, BalancedAccuracy: 89.38%, GeometricMean: 89.07%, CohenKappa: 81.08%
[3,000] Accuracy: 92.43%, BalancedAccuracy: 90.39%, GeometricMean: 90.17%, CohenKappa: 82.65%
[4,000] Accuracy: 93.05%, BalancedAccuracy: 91.22%, GeometricMean: 91.06%, CohenKappa: 83.99%
[5,000] Accuracy: 93.26%, BalancedAccuracy: 91.55%, GeometricMean: 91.41%, CohenKappa: 84.54%
[6,000] Accuracy: 87.45%, BalancedAccuracy: 86.54%, GeometricMean: 86.46%, CohenKappa: 73.21%
[7,000] Accuracy: 84.27%, BalancedAccuracy: 84.02%, GeometricMean: 84.01%, CohenKappa: 67.68%
[8,000] Accuracy: 82.37%, BalancedAccuracy: 82.40%, GeometricMean: 82.40%, CohenKappa: 64.39%
[9,000] Accuracy: 80.71%, BalancedAccuracy: 80.85%, GeometricMean: 80.84%, CohenKappa: 61.36%
[10,000] Accuracy: 79.03%, BalancedAccuracy: 79.22%, GeometricMean: 79.14%, CohenKappa: 58.14%
[11,000] Accuracy: 77.40%, BalancedAccuracy: 77.67%, Geomet

Accuracy: 80.08%, BalancedAccuracy: 80.16%, GeometricMean: 80.15%, CohenKappa: 60.17%

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.